Домашнее задание
ДЗ №2 «Первая модель - учимся оценивать качество»

Цель:
Студент научится строить конвейер тестирования торговой стратегии.
Результат - конвейер оценки качества торговой стратегии, готовый для дальнейшего использования при построении моделей.


Описание/Пошаговая инструкция выполнения домашнего задания:
1. Создать код на Python, который разделит ваши данные на тренировочный, тестовый и валидационный наборы данных.
2. Построить одну или несколько моделей на основе классификации, которая будет принимать торговые решения по бумагам SnP500 и/или криптовалютам.
3. Построить конвейер обратного тестирования полученной стратегии.
4. Провести подбор гиперпараметров моделей с использованием обучающей и тестовой выборок.
5. Провести финальное тестирование построенных торговых стратегий на валидационном наборе данных и сравнить их между собой.
6. Сформировать дашборд, показывающий эффективность различных стратегий во времени.

In [220]:
!pip install pandas numpy TA-Lib-Precompiled scikit-learn hyperopt plotly backtesting backtrader
!pip install matplotlib requests_cache
!pip install TA-Lib
!pip install yfinance_cache
!pip install fmp_python
!pip install outlier_utils
!pip install mplfinance

  Using cached TA-Lib-Precompiled-0.4.25.tar.gz (276 kB)
  Preparing metadata (setup.py) ... done
  Using cached backtesting-0.6.5-py3-none-any.whl.metadata (7.0 kB)
  Using cached backtrader-1.9.78.123-py2.py3-none-any.whl.metadata (6.8 kB)
Using cached backtesting-0.6.5-py3-none-any.whl (192 kB)
Using cached backtrader-1.9.78.123-py2.py3-none-any.whl (419 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for TA-Lib-Precompiled
  Running setup.py clean for TA-Lib-Precompiled
Failed to build TA-Lib-Precompiled
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (TA-Lib-Precompiled)


In [221]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import yfinance as yf
import plotly.express as px
import talib as ta
from sklearn.model_selection import train_test_split
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from statsmodels.tsa.seasonal import STL
import pywt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
import mplfinance as mpf
from scipy.fft import fft
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from IPython.display import display
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from statsmodels.tsa.seasonal import STL
import pywt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
import mplfinance as mpf
from scipy.fft import fft
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from IPython.display import display
import matplotlib.pyplot as plt

In [222]:
# Класс для базовой стратегии
class BaseStrategy:
    def __init__(self):
        self.positions = []
        self.equity = []

    def calculate_metrics(self, data):
        """Описание используемых метрик производительности:
           final_equity: Конечный капитал после завершения торгового периода. Показывает, насколько вырос или уменьшился начальный капитал в результате торговли.
           Используется для оценки общей эффективности стратегии. Чем выше конечный капитал, тем успешнее стратегия.
           n_trades: Количество сделок, совершенных за период тестирования. Показывает активность стратегии. Большое количество сделок может указывать на высокую частоту торговли, что может быть как преимуществом, так и недостатком в зависимости от контекста.
           returns: Общая доходность стратегии в процентах за период тестирования. Основной показатель прибыльности стратегии. Положительная доходность указывает на успешность стратегии, отрицательная — на убыточность.
           max_drawdown: Максимальная просадка капитала за период тестирования. Показывает наибольшее падение капитала от пика до минимума. Используется для оценки риска стратегии. Чем меньше просадка, тем стабильнее стратегия.
           win_rate: Процент прибыльных сделок от общего количества сделок. Показывает, насколько часто стратегия приносит прибыль. Высокий процент выигрышных сделок может указывать на стабильность стратегии.
           sharpe_ratio: Коэффициент Шарпа измеряет доходность с поправкой на риск. Чем выше значение, тем лучше стратегия с точки зрения доходности на единицу риска. Используется для сравнения стратегий с учетом риска. Высокий коэффициент Шарпа указывает на эффективное использование риска.
           sortino_ratio: Коэффициент Сортино, который измеряет доходность с поправкой на риск, учитывая только негативную волатильность. Аналогичен коэффициенту Шарпа, но фокусируется на риске убытков, что делает его более подходящим для оценки стратегий с асимметричным распределением доходности.
           profit_factor: Отношение общей прибыли к общему убытку. Показывает, насколько прибыльна стратегия по сравнению с убытками. Используется для оценки соотношения прибыли и убытков. Значение больше 1 указывает на прибыльность стратегии.
        """
        if not self.positions:
            return {
                'final_equity': 1.0,
                'n_trades': 0,
                'returns': 0,
                'max_drawdown': 0,
                'win_rate': 0,
                'sharpe_ratio': 0,
                'sortino_ratio': 0,
                'profit_factor': 0
            }

        # Расчет базовых метрик
        final_equity = self.equity[-1]
        n_trades = len(self.positions)
        returns = (final_equity - 1.0) * 100

        # Расчет максимальной просадки
        peak = 1.0
        drawdowns = []
        for eq in self.equity:
            if eq > peak:
                peak = eq
            drawdown = (peak - eq) / peak * 100
            drawdowns.append(drawdown)
        max_drawdown = max(drawdowns)

        # Расчет win rate
        profitable_trades = sum(1 for pos in self.positions if pos['profit'] > 0)
        win_rate = profitable_trades / n_trades * 100 if n_trades > 0 else 0

        # Расчет Sharpe и Sortino ratio
        daily_returns = pd.Series(self.equity).pct_change().dropna()
        if len(daily_returns) > 0:
            sharpe_ratio = np.sqrt(252) * daily_returns.mean() / daily_returns.std()
            downside_returns = daily_returns[daily_returns < 0]
            sortino_ratio = np.sqrt(252) * daily_returns.mean() / downside_returns.std() if len(downside_returns) > 0 else 0
        else:
            sharpe_ratio = sortino_ratio = 0

        # Расчет profit factor
        gross_profit = sum(pos['profit'] for pos in self.positions if pos['profit'] > 0)
        gross_loss = abs(sum(pos['profit'] for pos in self.positions if pos['profit'] < 0))
        profit_factor = gross_profit / gross_loss if gross_loss != 0 else 0

        return {
            'final_equity': final_equity,
            'n_trades': n_trades,
            'returns': returns,
            'max_drawdown': max_drawdown,
            'win_rate': win_rate,
            'sharpe_ratio': sharpe_ratio,
            'sortino_ratio': sortino_ratio,
            'profit_factor': profit_factor
        }

In [223]:
# Стратегия 4: Золотой крест
#https://sergeytereshkin.ru/library/zolotoy-krest-signal-razvorota-i-metodika-poiska-na-grafike
class Strategy4(BaseStrategy):
    """
    «Золотой крест» — один из самых надежных бычьих сигналов технического анализа,
    позволяющий трейдерам структурировать вход в длинную позицию при строгом риск-менеджменте и подтверждении индикаторами.

    Выбор и настройка скользящих средних
    Оптимальные периоды для SMA
     - Для дневных и недельных графиков оптимальным сочетанием является SMA50/SMA200, поскольку SMA сглаживает шум и отражает среднесрочную и долгосрочную динамику цены.

    Использование EMA на коротких таймфреймах
     - На коротких таймфреймах (M15–H1) предпочтительнее EMA50/EMA200: EMA быстрее реагирует на последние изменения цены, что позволяет оперативнее фиксировать разворот, но требует дополнительной фильтрации шума.

    Бэктестинг настроек MA
     - Оптимальные параметры следует подбирать через бэктестинг: тестировать разные комбинации периодов,
       фиксируя доходность и максимальную просадку каждой стратегии.

    Подтверждающие индикаторы и фильтры
     - MACD как индикатор тренда
     - MACD подтверждает силу бычьего сигнала при росте гистограммы выше нулевой линии и пересечении сигнальной линии снизу вверх,
       что усиливает вероятность продолжения восходящего движения.

    RSI и оценка перекупленности
     - RSI в диапазоне 50–70 указывает на адекватный бычий импульс без перегрева рынка и помогает отсеивать ложные сигналы «золотого креста» во время флета.

    Анализ объемов и свечных паттернов
     - Дополнительным фильтром служат объемы: рост торгового объема в момент пересечения показывает активное участие игроков и подтверждает значимость сигнала.
       Бычьи свечные формации, такие как поглощение или «утренняя звезда», рядом с точкой пересечения усиливают доверие к сигналу.

    Источник Yandex:
    Некоторые оптимальные параметры стратегии Golden Cross в трейдинге:
      - Первичные условия: пересечение MA(50) над MA(200) с закреплением выше на 2+ торговых дня.
      - Направленность скользящих средних: обе MA направлены вверх под углом не менее 5°.
      - Расстояние между MA: минимальный разрыв 1,5% для исключения случайных пересечений.
      - Объёмное подтверждение: превышение 20-дневного среднего объёма на 30%+.
      - Дополнительные фильтры качества:
         RSI(14) в диапазоне 45–70 (для исключения экстремальных состояний перекупленности),
         MACD выше нулевой линии (для подтверждения смены импульса на бычий),
         отсутствие сильных уровней сопротивления в ближайших 8–12% от цены пересечения,
         макроэкономический фон (отсутствие ожидаемых негативных событий в ближайшие 4–6 недель).

    """
    def __init__(self, bb_period=20, bb_std=2, rsi_period=14, sma=10, ema_period=4, ma_period=10, macd_fast=12, macd_slow=26, macd_signal=9):
        super().__init__()
        self.bb_period = int(bb_period)
        self.bb_std = float(bb_std)
        self.rsi_period = int(rsi_period)
        self.sma = int(sma)
        self.ema_period = int(ema_period)
        self.ma_period = int(ma_period)
        self.macd_fast = int(macd_fast)
        self.macd_slow = int(macd_slow)
        self.macd_signal = int(macd_signal)

    def get_params(self):
        """Возвращает текущие гиперпараметры стратегии"""
        return {
            'bb_period': self.bb_period,
            'bb_std': self.bb_std,
            'rsi_period': self.rsi_period,
            'sma': self.sma,
            'ema_period': self.ema_period,
            'ma_period': self.ma_period,
            'macd_fast': self.macd_fast,
            'macd_slow': self.macd_slow,
            'macd_signal': self.macd_signal
        }

    def generate_signals(self, data):
        try:
            df = data.copy()
            close_prices = df['Close'].values.astype(float).flatten()
            open_prices = df['Open'].values.astype(float).flatten()
            high_prices = df['High'].values.astype(float).flatten()
            low_prices = df['Low'].values.astype(float).flatten()
            volume = df['Volume'].values.astype(float).flatten()

            #Полосы Боллинджера - Уровни поддержки и сопротивления
            df['BB_middle'] = ta.SMA(close_prices, timeperiod=self.bb_period)
            std_dev = ta.STDDEV(close_prices, timeperiod=self.bb_period)
            df['BB_upper'] = df['BB_middle'] + self.bb_std * std_dev
            df['BB_lower'] = df['BB_middle'] - self.bb_std * std_dev

            df['sma'] = ta.SMA(close_prices, timeperiod=self.sma)
            df['ma_period'] = ta.MA(close_prices, timeperiod=self.ma_period)
            df['RSI'] = ta.RSI(close_prices, timeperiod=self.rsi_period)
            #df['obv'] = np.where(df['Close'] > df['Close'].shift(1), df['Volume'], np.where(df['Close'] < df['Close'].shift(1), -df['Volume'], 0)).cumsum()
            df['obv'] = ta.OBV(close_prices, volume)
            df['engulfing'] = ta.CDLENGULFING(open_prices, high_prices, low_prices, close_prices)
            df['morningstar'] = ta.CDLMORNINGSTAR(open_prices, high_prices, low_prices, close_prices)
            #df['adosc'] = ta.ADOSC(high_prices, low_prices, close_prices, volume)

            macd, signal, _ = ta.MACD(close_prices,
                              fastperiod=self.macd_fast,
                              slowperiod=self.macd_slow,
                              signalperiod=self.macd_signal)
            df['MACD'] = macd
            df['Signal'] = signal
            #df = df.fillna(method='ffill').fillna(method='bfill')
            df = df.ffill().copy()
            signals = pd.Series(index=df.index, data=0)
            long_condition = ((df['engulfing'] > 0) | (df['morningstar'] > 0)) #& ((df['Close'] < df['sma']) & (df['Close'].shift(1) >= df['sma'].shift(1))) & (df['MACD'] > df['Signal']) & (df['RSI'] > 50) & (df['RSI'] < 70) & (df['obv'] > 0)
            #long_condition =  (df['RSI'] > 50) & (df['sma_period'] > 50) & (df['Close'] > df['EMA']) & (df['MACD'] > df['Signal']) & (df['obv'] > 0)
            #short_condition =  (df['RSI'] < 70) & (df['sma_period'] < 200) & (df['Close'] < df['EMA']) & (df['MACD'] < df['Signal']) & (df['obv'] < 0)
            #short_condition = Стоп-лосс | тейк-профит | трейлинг-стоп
            short_condition =  (((df['sma'] < 50)) | ((df['Close'] < df['sma']) & (df['Close'].shift(1) >= df['sma'].shift(1)))) | ((df['RSI'] > 50) & (df['MACD'] < df['Signal'])) | (df['Close'] > df['BB_upper'])
            signals[long_condition] = 1
            signals[short_condition] = -1
            signals = signals.reindex_like(data)
            return signals
        except Exception as e:
            print(f"Ошибка в generate_signals для Strategy4: {str(e)}")
            return None

In [224]:
# Стратегия 3: RSI + Candlestick Patterns
class Strategy3(BaseStrategy):
    """
    Комбинированная стратегия разворота тренда, использующая:
    - RSI (Relative Strength Index) - для определения перекупленности/перепроданности
    - Паттерны свечей - для подтверждения разворота (Doji, Hammer, Engulfing)

    Бизнес-логика стратегии:
    1. Основной принцип:
       - Поиск точек разворота в перекупленных/перепроданных зонах
       - Использование паттернов свечей для подтверждения разворота

    2. Условия для входа в длинную позицию (Long):
       - RSI находится в зоне перепроданности (< 30)
       - Формируется один из бычьих паттернов:
         * Doji (свеча неопределенности)
         * Hammer (молот)
         * Bullish Engulfing (бычье поглощение)

    3. Условия для входа в короткую позицию (Short):
       - RSI находится в зоне перекупленности (> 70)
       - Формируется один из медвежьих паттернов:
         * Doji (свеча неопределенности)
         * Inverted Hammer (перевернутый молот)
         * Bearish Engulfing (медвежье поглощение)

    4. Управление рисками:
       - RSI фильтрует слабые сигналы
       - Требуется подтверждение паттернами свечей
       - Учитывается сила сигнала разворота

    Особенности настройки:
    - rsi_period: период RSI (по умолчанию 14)
    - Паттерны свечей имеют фиксированные правила идентификации

    Преимущества стратегии:
    - Сочетание технического индикатора и визуального анализа
    - Четкие правила входа и выхода
    - Возможность раннего обнаружения разворотов тренда
    """
    def __init__(self, rsi_period=14):
        super().__init__()
        self.rsi_period = int(rsi_period)

    def get_params(self):
        """Возвращает текущие гиперпараметры стратегии"""
        return {
            'rsi_period': self.rsi_period
        }

    def generate_signals(self, data):
        try:
            df = data.copy()
            close_prices = df['Close'].values.astype(float).flatten()
            open_prices = df['Open'].values.astype(float).flatten()
            high_prices = df['High'].values.astype(float).flatten()
            low_prices = df['Low'].values.astype(float).flatten()
            df['RSI'] = ta.RSI(close_prices, timeperiod=self.rsi_period)
            df['doji'] = ta.CDLDOJI(open_prices, high_prices, low_prices, close_prices)
            df['hammer'] = ta.CDLHAMMER(open_prices, high_prices, low_prices, close_prices)
            df['engulfing'] = ta.CDLENGULFING(open_prices, high_prices, low_prices, close_prices)
            df = df.ffill().copy()
            #df = df.fillna(method='ffill').fillna(method='bfill')
            signals = pd.Series(index=df.index, data=0)
            long_condition = ((df['doji'] > 0) | (df['hammer'] > 0) | (df['engulfing'] > 0)) & (df['RSI'] < 30)
            short_condition = ((df['doji'] < 0) | (df['hammer'] < 0) | (df['engulfing'] < 0)) & (df['RSI'] > 70)
            signals[long_condition] = 1
            signals[short_condition] = -1
            signals = signals.reindex_like(data)
            return signals
        except Exception as e:
            print(f"Ошибка в generate_signals для Strategy3: {str(e)}")
            return None

In [225]:
class Strategy2(BaseStrategy):
    """
    Стратегия торговли на отскок (Mean Reversion), использующая:
    - Bollinger Bands - для определения волатильности и экстремальных значений цены
    - Stochastic Oscillator - для подтверждения разворота

    Бизнес-логика стратегии:
    1. Основной принцип:
       - Торговля от краев полос Боллинджера к средней линии
       - Использование стохастика для подтверждения сигналов разворота

    2. Условия для входа в длинную позицию (Long):
       - Цена находится ниже нижней полосы Боллинджера (перепроданность)
       - Линия %K стохастика пересекает линию %D снизу вверх (подтверждение разворота вверх)

    3. Условия для входа в короткую позицию (Short):
       - Цена находится выше верхней полосы Боллинджера (перекупленность)
       - Линия %K стохастика пересекает линию %D сверху вниз (подтверждение разворота вниз)

    4. Управление рисками:
       - Полосы Боллинджера автоматически адаптируются к волатильности
       - Стохастик подтверждает потенциал разворота
       - Торговля против тренда только при сильных сигналах

    Особенности настройки:
    - bb_period: период Bollinger Bands (по умолчанию 20)
    - bb_std: количество стандартных отклонений (по умолчанию 2)
    - stoch_k/d/slow: параметры стохастика, влияющие на его чувствительность
    """
    def __init__(self, bb_period=20, bb_std=2, stoch_k=14, stoch_d=3, stoch_slow=3, rsi_period = 14):
        super().__init__()
        self.bb_period = int(bb_period)
        self.bb_std = float(bb_std)
        self.stoch_k = int(stoch_k)
        self.stoch_d = int(stoch_d)
        self.stoch_slow = int(stoch_slow)
        self.rsi_period = int(rsi_period)

    def get_params(self):
        """Возвращает текущие гиперпараметры стратегии"""
        return {
            'bb_period': self.bb_period,
            'bb_std': self.bb_std,
            'stoch_k': self.stoch_k,
            'stoch_d': self.stoch_d,
            'stoch_slow': self.stoch_slow,
            'rsi_period': self.rsi_period
        }

    def generate_signals(self, data):
        try:
            df = data.copy()
            close_prices = df['Close'].values.astype(float).flatten()
            high_prices = df['High'].values.astype(float).flatten()
            low_prices = df['Low'].values.astype(float).flatten()
            df['BB_middle'] = ta.SMA(close_prices, timeperiod=self.bb_period)
            std_dev = ta.STDDEV(close_prices, timeperiod=self.bb_period)
            df['BB_upper'] = df['BB_middle'] + self.bb_std * std_dev
            df['BB_lower'] = df['BB_middle'] - self.bb_std * std_dev
            df['RSI'] = ta.RSI(close_prices, timeperiod=self.rsi_period)
            slowk, slowd = ta.STOCH(high_prices,
                                   low_prices,
                                   close_prices,
                                   fastk_period=self.stoch_k,
                                   slowk_period=self.stoch_d,
                                   slowd_period=self.stoch_slow)
            df['slowk'] = slowk
            df['slowd'] = slowd
            df = df.ffill().copy()
            #df = df.fillna(method='ffill').fillna(method='bfill')
            signals = pd.Series(index=df.index, data=0)
            long_condition = (df['Close'] < df['BB_lower']) & (df['slowk'] > df['slowd']) & (df['RSI'] < 70)
            short_condition = (df['Close'] > df['BB_upper']) & (df['slowk'] < df['slowd']) & (df['RSI'] > 30)
            signals[long_condition] = 1
            signals[short_condition] = -1
            signals = signals.reindex_like(data)
            return signals
        except Exception as e:
            print(f"Ошибка в generate_signals для Strategy2: {str(e)}")
            return None




In [226]:
class Strategy1(BaseStrategy):
    """
    Комбинированная трендово-импульсная стратегия, использующая:
    - EMA (Exponential Moving Average) - для определения тренда
    - RSI (Relative Strength Index) - для определения перекупленности/перепроданности
    - MACD (Moving Average Convergence Divergence) - для определения моментума и точек входа/выхода

    Бизнес-логика стратегии:
    1. Определение тренда:
       - Цена выше EMA = восходящий тренд
       - Цена ниже EMA = нисходящий тренд

    2. Условия для входа в длинную позицию (Long):
       - Цена находится выше EMA (подтверждение восходящего тренда)
       - RSI < 70 (нет перекупленности)
       - MACD пересекает Signal Line снизу вверх (сигнал усиления восходящего импульса)

    3. Условия для входа в короткую позицию (Short):
       - Цена находится ниже EMA (подтверждение нисходящего тренда)
       - RSI > 30 (нет перепроданности)
       - MACD пересекает Signal Line сверху вниз (сигнал усиления нисходящего импульса)

    4. Управление рисками:
       - RSI используется как фильтр для предотвращения входа в перекупленной/перепроданной зоне
       - EMA подтверждает направление тренда
       - MACD подтверждает силу импульса

    Особенности настройки:
    - ema_period: период EMA (по умолчанию 20) - чем больше период, тем более долгосрочный тренд отслеживается
    - rsi_period: период RSI (по умолчанию 14) - влияет на чувствительность индикатора к изменениям цены
    - macd_fast/slow/signal: периоды MACD - влияют на скорость реакции на изменение тренда
    """
    def __init__(self, ema_period=20, rsi_period=14, macd_fast=12, macd_slow=26, macd_signal=9):
        super().__init__()
        self.ema_period = int(ema_period)
        self.rsi_period = int(rsi_period)
        self.macd_fast = int(macd_fast)
        self.macd_slow = int(macd_slow)
        self.macd_signal = int(macd_signal)

    def get_params(self):
        """Возвращает текущие гиперпараметры стратегии"""
        return {
            'ema_period': self.ema_period,
            'rsi_period': self.rsi_period,
            'macd_fast': self.macd_fast,
            'macd_slow': self.macd_slow,
            'macd_signal': self.macd_signal
        }

    def generate_signals(self, data):
        try:
            df = data.copy()
            close_prices = df['Close'].values.astype(float).flatten()
            df['EMA'] = ta.EMA(close_prices, timeperiod=self.ema_period)
            df['RSI'] = ta.RSI(close_prices, timeperiod=self.rsi_period)
            macd, signal, _ = ta.MACD(close_prices,
                                      fastperiod=self.macd_fast,
                                      slowperiod=self.macd_slow,
                                      signalperiod=self.macd_signal)
            df['MACD'] = macd
            df['Signal'] = signal
            df = df.ffill().copy()
            #df = df.fillna(method='ffill').fillna(method='bfill')
            signals = pd.Series(index=df.index, data=0)
            long_condition = (df['Close'] > df['EMA']) & (df['RSI'] < 70) & (df['MACD'] > df['Signal'])
            short_condition = (df['Close'] < df['EMA']) & (df['RSI'] > 30) & (df['MACD'] < df['Signal'])
            signals[long_condition] = 1
            signals[short_condition] = -1
            signals = signals.reindex_like(data)
            return signals
        except Exception as e:
            print(f"Ошибка в generate_signals для Strategy1: {str(e)}")
            return None


In [227]:
# Определение пространств параметров для оптимизации
param_spaces = {
    'Strategy1': {
        'ema_period': hp.quniform('ema_period', 10, 50, 1),
        'rsi_period': hp.quniform('rsi_period', 7, 21, 1),
        'macd_fast': hp.quniform('macd_fast', 8, 20, 1),
        'macd_slow': hp.quniform('macd_slow', 21, 40, 1),
        'macd_signal': hp.quniform('macd_signal', 5, 15, 1)
    },
    'Strategy2': {
        'bb_period': hp.quniform('bb_period', 10, 50, 1),
        'bb_std': hp.uniform('bb_std', 1.5, 3.0),
        'stoch_k': hp.quniform('stoch_k', 7, 21, 1),
        'stoch_d': hp.quniform('stoch_d', 3, 7, 1),
        'stoch_slow': hp.quniform('stoch_slow', 3, 7, 1),
        'rsi_period': hp.quniform('rsi_period', 7, 21, 1)
    },
    'Strategy3': {
        'rsi_period': hp.quniform('rsi_period', 7, 21, 1)
    },
    'Strategy4': {
        'bb_period': hp.quniform('bb_period', 10, 50, 1),
        'bb_std': hp.uniform('bb_std', 1.5, 3.0),
        'rsi_period': hp.quniform('rsi_period', 7, 21, 1),
        'sma': hp.quniform('sma', 10, 50, 1),
        'ema_period': hp.quniform('ema_period', 10, 50, 1),
        'ma_period': hp.quniform('ma_period', 10, 50, 1),
        'macd_fast': hp.quniform('macd_fast', 8, 20, 1),
        'macd_slow': hp.quniform('macd_slow', 21, 40, 1),
        'macd_signal': hp.quniform('macd_signal', 5, 15, 1)
    }
}

In [228]:
# Функция для бэктестирования стратегии
def backtest_strategy(strategy, data):
    """Бэктестирование стратегии"""
    try:
        if data is None or len(data) < 2:
            return None

        signals = strategy.generate_signals(data)
        if signals is None or signals.empty:
            return None

        signals = signals.fillna(0)
        signals = signals.reindex(data.index, fill_value=0)

        close_prices = data['Close'].values
        equity = 1.0
        equity_curve = [equity]
        positions = []
        position_open = False
        entry_price = 0
        entry_signal = 0
        entry_date = None

        for i in range(1, len(signals)):
            current_signal = signals.iloc[i]
            current_date = signals.index[i]

            if not position_open and current_signal != 0:
                # Открываем позицию
                position_open = True
                entry_price = close_prices[i]
                entry_signal = current_signal
                entry_date = current_date

            elif position_open and ((current_signal != 0 and current_signal != entry_signal) or i == len(signals)-1):
                # Закрываем позицию
                exit_price = close_prices[i]
                profit = (exit_price - entry_price) / entry_price * entry_signal

                positions.append({
                    'entry_date': entry_date,
                    'exit_date': current_date,
                    'profit': float(profit)
                })

                equity *= (1.0 + profit)
                position_open = False

            equity_curve.append(float(equity))

        strategy.positions = positions
        strategy.equity = equity_curve
        strategy.signals = signals  # Сохраняем сигналы для последующей визуализации

        return strategy.calculate_metrics(data)

    except Exception as e:
        print(f"Ошибка при бэктестировании: {str(e)}")
        return None

In [229]:
# Функция для бэктестирования Buy & Hold стратегии
def backtest_buy_hold(data):
    """Бэктестирование Buy & Hold стратегии"""
    initial_price = float(data['Close'].iloc[0])
    final_price = float(data['Close'].iloc[-1])
    returns = (final_price - initial_price) / initial_price * 100

    daily_returns = data['Close'].pct_change().dropna()

    if len(daily_returns) > 0:
        sharpe_ratio = np.sqrt(252) * daily_returns.mean() / daily_returns.std()
        downside_returns = daily_returns[daily_returns < 0]
        sortino_ratio = np.sqrt(252) * daily_returns.mean() / downside_returns.std() if len(downside_returns) > 0 else 0
    else:
        sharpe_ratio = sortino_ratio = 0

    rolling_max = data['Close'].expanding().max()
    drawdowns = (rolling_max - data['Close']) / rolling_max
    max_drawdown = float(drawdowns.max() * 100)

    return {
        'final_equity': 1 + returns / 100,
        'n_trades': 1,
        'returns': returns,
        'max_drawdown': max_drawdown,
        'win_rate': 100.0 if returns > 0 else 0.0,
        'sharpe_ratio': float(sharpe_ratio),
        'sortino_ratio': float(sortino_ratio),
        'profit_factor': float('inf') if returns > 0 else 0.0
    }


In [230]:
def optimize_strategy(strategy_class, data, param_space):
    def objective(params):
        strategy = strategy_class(**params)
        metrics = backtest_strategy(strategy, data)
        if metrics:
            return {'loss': -metrics['profit_factor'], 'status': STATUS_OK}
        else:
            return {'loss': 0, 'status': STATUS_OK}

    trials = Trials()
    best = fmin(fn=objective,
                space=param_space,
                algo=tpe.suggest,
                max_evals=50,
                trials=trials)

    return best

In [231]:
def format_metric(value):
    """Форматирует число с двумя знаками после точки, если есть дробная часть"""
    if isinstance(value, (int, float)):
        return round(value, 2) if not value.is_integer() else int(value)
    return value

In [232]:
# Функция для создания сводной таблицы результатов
def create_summary_table(results, instrument):
    """Создание сводной таблицы результатов"""
    metrics = ['final_equity', 'n_trades', 'returns', 'max_drawdown',
               'win_rate', 'sharpe_ratio', 'sortino_ratio', 'profit_factor']

    # Создаем MultiIndex для более структурированного отображения
    index_tuples = []
    data = []

    # Buy & Hold результаты
    if 'Buy & Hold' in results and 'full' in results['Buy & Hold']:
        bh_metrics = results['Buy & Hold']['full']
        index_tuples.append(('Buy & Hold', 'Full'))
        data.append({
            'Days': len(results['Buy & Hold']['full_data']),
            'Hyperparameters': 'N/A',
            **{metric: format_metric(bh_metrics[metric]) for metric in metrics}
        })

    # Результаты торговых стратегий
    #for strategy in ['Strategy2', 'Strategy3', 'Strategy1', 'Strategy4']:
    for strategy in ['Strategy4']:
        if strategy in results:
            for dataset in ['train_default', 'test', 'valid']:
                if dataset in results[strategy]:
                    metrics_data = results[strategy][dataset]

                    # Определяем, какие гиперпараметры использовать
                    if dataset == 'train_default':
                        hyperparams = results[strategy].get('train_default_params', 'N/A')
                    elif dataset == 'test':
                        hyperparams = results[strategy].get('optimized_params', 'N/A')
                    elif dataset == 'valid':
                        # Для Valid используем гиперпараметры, оптимизированные на объединенном наборе Train и Test
                        hyperparams = results[strategy].get('optimized_params_combined', 'N/A')

                    days = len(results[strategy][f'{dataset}_data'])

                    # Заменяем "train_default" на "Train"
                    dataset_name = 'Train' if dataset == 'train_default' else dataset.capitalize()

                    # Форматируем гиперпараметры с двумя знаками после точки, если есть дробная часть
                    formatted_hyperparams = {}
                    if hyperparams != 'N/A':
                        for param, value in hyperparams.items():
                            formatted_hyperparams[param] = round(value, 2) if isinstance(value, float) else value
                    else:
                        formatted_hyperparams = 'N/A'

                    index_tuples.append((strategy, dataset_name))
                    data.append({
                        'Days': days,
                        'Hyperparameters': formatted_hyperparams,
                        **{metric: format_metric(metrics_data[metric]) for metric in metrics}
                    })

    # Создаем DataFrame с MultiIndex
    df = pd.DataFrame(data, index=pd.MultiIndex.from_tuples(index_tuples, names=['Strategy', 'Dataset']))

    # Форматируем таблицу для красивого отображения
    styled_df = df.style\
        .format({
            'final_equity': '{:.2f}',
            'returns': '{:.2f}%',
            'max_drawdown': '{:.2f}%',
            'win_rate': '{:.2f}%',
            'sharpe_ratio': '{:.2f}',
            'sortino_ratio': '{:.2f}',
            'profit_factor': '{:.2f}'
        })\
        .set_caption(f'Сводная таблица результатов бэктестирования стратегий для {instrument}')\
        .set_table_styles([
            {'selector': 'caption', 'props': [('text-align', 'center'), ('font-size', '16px'), ('font-weight', 'bold')]},
            {'selector': 'th', 'props': [('text-align', 'center'), ('background-color', '#f0f0f0')]},
            {'selector': 'td', 'props': [('text-align', 'right')]}
        ])\
        .background_gradient(subset=['profit_factor'], cmap='RdYlGn')\
        .background_gradient(subset=['returns'], cmap='RdYlGn')

    return styled_df

In [233]:
def analyze_trades(strategy, data, signals):
    """ Анализ фактических сделок на основе сигналов и ценовых данных. Возвращает список сделок с точками входа/выхода и прибылью. """
    trades = []
    in_position = False
    entry_price = 0
    entry_date = None
    position_type = None  # 'long' or 'short'

    for i in range(len(data)):
        current_date = data.index[i]
        current_price = data['Close'].iloc[i]
        current_signal = signals.iloc[i]

        if not in_position and current_signal != 0:
            # Opening a position
            in_position = True
            entry_price = current_price
            entry_date = current_date
            position_type = 'long' if current_signal == 1 else 'short'

        elif in_position:
            # Check for closing conditions
            should_close = (
                (position_type == 'long' and current_signal == -1) or
                (position_type == 'short' and current_signal == 1) or
                (i == len(data) - 1)  # Close at the end of the period
            )

            if should_close:
                # Calculate profit
                profit = (
                    (current_price - entry_price) / entry_price * 100
                    if position_type == 'long'
                    else (entry_price - current_price) / entry_price * 100
                )

                trades.append({
                    'entry_date': entry_date,
                    'entry_price': entry_price,
                    'exit_date': current_date,
                    'exit_price': current_price,
                    'type': position_type,
                    'profit': profit
                })

                in_position = False
                entry_price = 0
                entry_date = None
                position_type = None

    return trades

In [234]:
# Функция оценки стратегических показателей и расчета доходности с поправкой на риск
def calculate_risk_adjusted_returns(metrics):
    """Risk-Adjusted Returns позволяет выбрать стратегию, которая приносит доходность с минимальным риском..
       Он позволяет сравнивать стратегии не только по их абсолютной доходности, но и по тому, насколько эффективно они используют риск для достижения этой доходности.
       Оценить, насколько стратегия эффективно использует риск для достижения доходности. Это позволяет сравнивать стратегии, которые могут иметь схожую доходность, но разный уровень риска.
       Компоненты формулы: returns: Общая доходность стратегии. Чем выше доходность, тем выше значение метрики.
                           sharpe_ratio: Коэффициент Шарпа, который измеряет доходность с поправкой на общий риск (волатильность). Чем выше коэффициент Шарпа, тем лучше стратегия использует риск для получения доходности.
                           sortino_ratio: Коэффициент Сортино, который измеряет доходность с поправкой на риск убытков (негативную волатильность). Чем выше коэффициент Сортино, тем лучше стратегия управляет риском убытков.
                           profit_factor: Отношение общей прибыли к общему убытку. Чем выше значение, тем больше прибыль по сравнению с убытками.
                           max_drawdown: Максимальная просадка капитала. Чем меньше просадка, тем стабильнее стратегия. В формуле используется 1 + max_drawdown, чтобы избежать деления на ноль и уменьшить влияние больших просадок.
        Логика расчета: В числителе стоит произведение доходности, коэффициента Шарпа, коэффициента Сортино и фактора прибыли. Это позволяет учитывать как абсолютную доходность, так и эффективность управления риском.
                        В знаменателе - метрика максимальной просади, что снижает значение метрики, если стратегия имеет высокий уровень риска (большие просадки).
    """
    if not metrics:
        return float('-inf')

    # Нормализуем доходность по показателям риска
    risk_adjusted_return = (
        metrics['returns'] *
        metrics['sharpe_ratio'] *
        metrics['sortino_ratio'] *
        metrics['profit_factor']
    ) / (1 + metrics['max_drawdown'])  # Чтобы избежать деления на ноль.

    return risk_adjusted_return

In [235]:
def find_best_strategy(all_results):
    """ Поиск лучшей стратегии среди всех инструментов и наборов данных на основе доходности с поправкой на риск. """
    best_strategy = {
        'strategy_name': None,
        'instrument': None,
        'dataset': None,
        'metrics': None,
        'risk_adjusted_returns': float('-inf'),
        'params': None
    }

    for instrument, results in all_results.items():
        #for strategy_name in ['Strategy1', 'Strategy2', 'Strategy3', 'Strategy4']:
        for strategy_name in ['Strategy4']:
            if strategy_name not in results:
                continue

            # Check all datasets (train_default, test, valid)
            for dataset in ['train_default', 'test', 'valid']:
                if dataset not in results[strategy_name]:
                    continue

                metrics = results[strategy_name][dataset]
                risk_adjusted_returns = calculate_risk_adjusted_returns(metrics)

                if risk_adjusted_returns > best_strategy['risk_adjusted_returns']:
                    params_key = 'train_default_params' if dataset == 'train_default' else 'optimized_params'
                    if dataset == 'valid':
                        params_key = 'optimized_params_combined'

                    best_strategy = {
                        'strategy_name': strategy_name,
                        'instrument': instrument,
                        'dataset': dataset,
                        'metrics': metrics,
                        'risk_adjusted_returns': risk_adjusted_returns,
                        'params': results[strategy_name].get(params_key)
                    }

    return best_strategy

In [236]:
def plot_best_strategy(data, strategy, signals, equity_curve, strategy_results, trades, title):
    """ Функция визуализации, показывающая реальные сделки стратегии вместо сигналов """
    fig = make_subplots(rows=2, cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.03,
                        row_heights=[0.7, 0.3],
                        subplot_titles=(
                            "Price and Trades",  # Убираем дублирование заголовка
                            "Equity Curve"
                        ))

    # Candlestick chart
    fig.add_trace(
        go.Candlestick(
            x=data.index,
            open=data['Open'],
            high=data['High'],
            low=data['Low'],
            close=data['Close'],
            name='Price'
        ),
        row=1, col=1
    )

    # Отображение сделок
    long_entries = []
    long_exits = []
    short_entries = []
    short_exits = []

    for trade in trades:
        if trade['type'] == 'long':
            long_entries.append({
                'date': trade['entry_date'],
                'price': trade['entry_price'],
                'profit': trade['profit']
            })
            long_exits.append({
                'date': trade['exit_date'],
                'price': trade['exit_price'],
                'profit': trade['profit']
            })
        else:  # short trade
            short_entries.append({
                'date': trade['entry_date'],
                'price': trade['entry_price'],
                'profit': trade['profit']
            })
            short_exits.append({
                'date': trade['exit_date'],
                'price': trade['exit_price'],
                'profit': trade['profit']
            })

    # Отображение сделок long
    if long_entries:
        fig.add_trace(
            go.Scatter(
                x=[t['date'] for t in long_entries],
                y=[t['price'] * 0.99 for t in long_entries],
                mode='markers',
                marker=dict(
                    symbol='triangle-up',
                    size=12,
                    color='green',
                    line=dict(width=2)
                ),
                name='Long Entry',
                text=[f"Long Entry<br>Date: {t['date'].strftime('%Y-%m-%d')}<br>Price: {t['price']:.2f}<br>Profit: {t['profit']:.2f}%"
                      for t in long_entries],
                hoverinfo='text'
            ),
            row=1, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=[t['date'] for t in long_exits],
                y=[t['price'] * 1.01 for t in long_exits],
                mode='markers',
                marker=dict(
                    symbol='triangle-down',
                    size=12,
                    color='blue',
                    line=dict(width=2)
                ),
                name='Long Exit',
                text=[f"Long Exit<br>Date: {t['date'].strftime('%Y-%m-%d')}<br>Price: {t['price']:.2f}<br>Profit: {t['profit']:.2f}%"
                      for t in long_exits],
                hoverinfo='text'
            ),
            row=1, col=1
        )

    # Отображение сделок Short
    if short_entries:
        fig.add_trace(
            go.Scatter(
                x=[t['date'] for t in short_entries],
                y=[t['price'] * 1.01 for t in short_entries],
                mode='markers',
                marker=dict(
                    symbol='triangle-down',
                    size=12,
                    color='red',
                    line=dict(width=2)
                ),
                name='Short Entry',
                text=[f"Short Entry<br>Date: {t['date'].strftime('%Y-%m-%d')}<br>Price: {t['price']:.2f}<br>Profit: {t['profit']:.2f}%"
                      for t in short_entries],
                hoverinfo='text'
            ),
            row=1, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=[t['date'] for t in short_exits],
                y=[t['price'] * 0.99 for t in short_exits],
                mode='markers',
                marker=dict(
                    symbol='triangle-up',
                    size=12,
                    color='orange',
                    line=dict(width=2)
                ),
                name='Short Exit',
                text=[f"Short Exit<br>Date: {t['date'].strftime('%Y-%m-%d')}<br>Price: {t['price']:.2f}<br>Profit: {t['profit']:.2f}%"
                      for t in short_exits],
                hoverinfo='text'
            ),
            row=1, col=1
        )

    # Equity curve
    if isinstance(equity_curve, list):
        equity_curve = np.array(equity_curve)

    # Проверка соответствия equity_curve длине набора данных
    if len(equity_curve) > len(data):
        equity_curve = equity_curve[-len(data):]
    elif len(equity_curve) < len(data):
        padding = np.full(len(data) - len(equity_curve), equity_curve[-1])
        equity_curve = np.concatenate([equity_curve, padding])

    fig.add_trace(
        go.Scatter(
            x=data.index,
            y=equity_curve,
            name='Equity',
            line=dict(color='blue', width=2),
            text=[f"Equity: {eq:.2f}" for eq in equity_curve],
            hoverinfo='text+x'
        ),
        row=2, col=1
    )

    # Настройки графика
    fig.update_layout(
        title=dict(
            text=title,  # Заголовок передается только здесь
            x=0.5,
            xanchor='center'
        ),
        showlegend=True,
        height=800,
        template='plotly_white',
        hovermode='x unified',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    # Настройки осей графика
    fig.update_xaxes(
        title_text=" ",
        rangeslider_visible=False,
        gridcolor='lightgrey',
        showgrid=True
    )

    fig.update_yaxes(
        title_text="Price",
        gridcolor='lightgrey',
        showgrid=True,
        row=1, col=1
    )

    fig.update_yaxes(
        title_text="Equity",
        gridcolor='lightgrey',
        showgrid=True,
        row=2, col=1
    )

    return fig

In [237]:
# Функция визуализации результатов лучшей стратегии с сигналами
def plot_best_strategy_signals(data, strategy, signals, strategy_results, title1):
    """Функция визуализации результатов лучшей стратегии с отдельными подграфиками для каждого индикатора"""
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.droplevel(1)

    # Определяем количество подграфиков в зависимости от стратегии
    if isinstance(strategy, Strategy1):
        # Для Strategy1: EMA, RSI, MACD
        num_subplots = 4  # Основной график + 3 индикатора
        subplot_titles = ["Price и сигналы", "EMA", "RSI", "MACD"]  # Убираем дублирование заголовка
    elif isinstance(strategy, Strategy2):
        # Для Strategy2: Bollinger Bands, Stochastic + RSI
        num_subplots = 4  # Основной график + 3 индикатора
        subplot_titles = ["Price и сигналы", "Bollinger Bands", "Stochastic %K", "Stochastic %D", "RSI"]  # Убираем дублирование заголовка
    elif isinstance(strategy, Strategy3):
        # Для Strategy3: RSI
        num_subplots = 2  # Основной график + 1 индикатор
        subplot_titles = ["Price и сигналы", "RSI"]  # Убираем дублирование заголовка
    elif isinstance(strategy, Strategy4):
        # Для Strategy4: RSI
        num_subplots = 5  # Основной график + 5 индикаторов
        subplot_titles = ["Price и сигналы", "RSI", "EMA", "MACD", "Bollinger Bands"]  # Убираем дублирование заголовка
    else:
        num_subplots = 2  # По умолчанию: основной график + 1 индикатор
        subplot_titles = ["Price и сигналы", "Indicator"]  # Убираем дублирование заголовка

    # Создание subplot с несколькими графиками
    fig = make_subplots(rows=num_subplots, cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.05,
                        row_heights=[0.6] + [0.1] * (num_subplots - 1),  # Основной график занимает больше места
                        subplot_titles=subplot_titles)  # Заголовки для каждого подграфика

    # Добавление свечного графика на первый подграфик
    fig.add_trace(
        go.Candlestick(
            x=data.index,
            open=data['Open'],
            high=data['High'],
            low=data['Low'],
            close=data['Close'],
            name='Цена'
        ),
        row=1, col=1
    )

    # Добавление сигналов на первый подграфик
    buy_signals = signals[signals == 1].index
    sell_signals = signals[signals == -1].index

    # Покупки
    if len(buy_signals) > 0:
        fig.add_trace(
            go.Scatter(
                x=buy_signals,
                y=data.loc[buy_signals, 'Low'] * 0.99,
                mode='markers',
                marker=dict(
                    symbol='triangle-up',
                    size=12,
                    color='green',
                    line=dict(width=2)
                ),
                name='Сигнал на покупку',
                text=[f"Покупка<br>Дата: {date.strftime('%Y-%m-%d')}<br>Цена: {data.loc[date, 'Close']:.2f}"
                      for date in buy_signals],
                hoverinfo='text'
            ),
            row=1, col=1
        )

    # Продажи
    if len(sell_signals) > 0:
        fig.add_trace(
            go.Scatter(
                x=sell_signals,
                y=data.loc[sell_signals, 'High'] * 1.01,
                mode='markers',
                marker=dict(
                    symbol='triangle-down',
                    size=12,
                    color='red',
                    line=dict(width=2)
                ),
                name='Сигнал на продажу',
                text=[f"Продажа<br>Дата: {date.strftime('%Y-%m-%d')}<br>Цена: {data.loc[date, 'Close']:.2f}"
                      for date in sell_signals],
                hoverinfo='text'
            ),
            row=1, col=1
        )

    # Добавление индикаторов на отдельные подграфики
    if isinstance(strategy, Strategy1):
        # Для Strategy1: EMA, RSI, MACD
        close_prices = data['Close'].values.astype(float).flatten()
        df = data.copy()
        df['EMA'] = ta.EMA(close_prices, timeperiod=strategy.ema_period)
        df['RSI'] = ta.RSI(close_prices, timeperiod=strategy.rsi_period)
        macd, signal, _ = ta.MACD(close_prices,
                                  fastperiod=strategy.macd_fast,
                                  slowperiod=strategy.macd_slow,
                                  signalperiod=strategy.macd_signal)
        df['MACD'] = macd
        df['Signal'] = signal

        # EMA на втором подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['EMA'],
                name='EMA',
                line=dict(color='blue', width=1)
            ),
            row=2, col=1
        )

        # RSI на третьем подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['RSI'],
                name='RSI',
                line=dict(color='orange', width=1)
            ),
            row=3, col=1
        )

        # MACD и Signal на четвертом подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['MACD'],
                name='MACD',
                line=dict(color='green', width=1)
            ),
            row=4, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['Signal'],
                name='MACD Signal',
                line=dict(color='red', width=1)
            ),
            row=4, col=1
        )

    elif isinstance(strategy, Strategy2):
        # Для Strategy2: Bollinger Bands, Stochastic + RSI
        close_prices = data['Close'].values.astype(float).flatten()
        high_prices = data['High'].values.astype(float).flatten()
        low_prices = data['Low'].values.astype(float).flatten()
        df = data.copy()
        df['BB_middle'] = ta.SMA(close_prices, timeperiod=strategy.bb_period)
        std_dev = ta.STDDEV(close_prices, timeperiod=strategy.bb_period)
        df['RSI'] = ta.RSI(close_prices, timeperiod=strategy.rsi_period)
        df['BB_upper'] = df['BB_middle'] + strategy.bb_std * std_dev
        df['BB_lower'] = df['BB_middle'] - strategy.bb_std * std_dev
        slowk, slowd = ta.STOCH(high_prices,
                               low_prices,
                               close_prices,
                               fastk_period=strategy.stoch_k,
                               slowk_period=strategy.stoch_d,
                               slowd_period=strategy.stoch_slow)
        df['slowk'] = slowk
        df['slowd'] = slowd

        # Bollinger Bands на втором подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['BB_upper'],
                name='BB Upper',
                line=dict(color='blue', width=1)
            ),
            row=2, col=1
        )

        # RSI на третьем подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['RSI'],
                name='RSI',
                line=dict(color='orange', width=1)
            ),
            row=3, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['BB_middle'],
                name='BB Middle',
                line=dict(color='green', width=1)
            ),
            row=2, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['BB_lower'],
                name='BB Lower',
                line=dict(color='red', width=1)
            ),
            row=2, col=1
        )

        # Stochastic на третьем подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['slowk'],
                name='Stochastic %K',
                line=dict(color='purple', width=1)
            ),
            row=3, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['slowd'],
                name='Stochastic %D',
                line=dict(color='orange', width=1)
            ),
            row=3, col=1
        )


    elif isinstance(strategy, Strategy3):
        # Для Strategy3: RSI
        close_prices = data['Close'].values.astype(float).flatten()
        df = data.copy()
        df['RSI'] = ta.RSI(close_prices, timeperiod=strategy.rsi_period)

        # RSI на втором подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['RSI'],
                name='RSI',
                line=dict(color='blue', width=1)
            ),
            row=2, col=1
        )
    elif isinstance(strategy, Strategy4):
        # Для Strategy4
        close_prices = data['Close'].values.astype(float).flatten()
        df = data.copy()
        df['RSI'] = ta.RSI(close_prices, timeperiod=strategy.rsi_period)
        df['BB_middle'] = ta.SMA(close_prices, timeperiod=strategy.bb_period)
        std_dev = ta.STDDEV(close_prices, timeperiod=strategy.bb_period)
        df['BB_upper'] = df['BB_middle'] + strategy.bb_std * std_dev
        df['BB_lower'] = df['BB_middle'] - strategy.bb_std * std_dev
        df['EMA'] = ta.EMA(close_prices, timeperiod=strategy.ema_period)
        macd, signal, _ = ta.MACD(close_prices,
                                  fastperiod=strategy.macd_fast,
                                  slowperiod=strategy.macd_slow,
                                  signalperiod=strategy.macd_signal)
        df['MACD'] = macd
        df['Signal'] = signal


        # RSI на втором подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['RSI'],
                name='RSI',
                line=dict(color='blue', width=1)
            ),
            row=2, col=1
        )

        # Bollinger Bands на пятом подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['BB_upper'],
                name='BB Upper',
                line=dict(color='blue', width=1)
            ),
            row=5, col=1
        )

        # EMA на втором подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['EMA'],
                name='EMA',
                line=dict(color='blue', width=1)
            ),
            row=3, col=1
        )

        # MACD и Signal на четвертом подграфике
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['MACD'],
                name='MACD',
                line=dict(color='green', width=1)
            ),
            row=4, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['Signal'],
                name='MACD Signal',
                line=dict(color='red', width=1)
            ),
            row=4, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['BB_middle'],
                name='BB Middle',
                line=dict(color='green', width=1)
            ),
            row=5, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=df['BB_lower'],
                name='BB Lower',
                line=dict(color='red', width=1)
            ),
            row=5, col=1
        )

    # Настройка макета
    fig.update_layout(
        title=dict(
            text=title1,  # Заголовок передается только здесь
            x=0.5,
            xanchor='center'
        ),
        showlegend=True,
        height=200 * num_subplots,  # Высота графика зависит от количества подграфиков
        template='plotly_white',
        hovermode='x unified',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    # Настройка осей
    fig.update_xaxes(
        title_text=" ",
        rangeslider_visible=False,
        gridcolor='lightgrey',
        showgrid=True
    )

    fig.update_yaxes(
        title_text=" ",
        gridcolor='lightgrey',
        showgrid=True,
        row=1, col=1
    )

    # Настройка осей для индикаторов
    for i in range(2, num_subplots + 1):
        fig.update_yaxes(
            title_text=" ",
            gridcolor='lightgrey',
            showgrid=True,
            row=i, col=1
        )

    return fig

In [238]:
def analyze_instrument(symbol):
    """Анализ торговых стратегий для одного инструмента"""

    try:
        data = load_data(symbol)
        if data is None or len(data) < 100:
            print(f"Недостаточно данных для {symbol}")
            return None, None

        data = data.copy()
        data = data.ffill().copy()
        #data = data.fillna(method='ffill').fillna(method='bfill')

        if data.isnull().any().any():
            print(f"Обнаружены пропущенные значения после обработки в {symbol}")
            return None, None

        print(f"\nРазмер данных: {len(data)} строк")

        train_data, test_data, valid_data = split_data(data)

        print("\nПериоды наборов данных:")
        print(f"Train: {train_data.index[0].date()} - {train_data.index[-1].date()}")
        print(f"Test: {test_data.index[0].date()} - {test_data.index[-1].date()}")
        print(f"Valid: {valid_data.index[0].date()} - {valid_data.index[-1].date()}")

        results = {}

        try:
            bh_results = backtest_buy_hold(data)
            if bh_results:
                results['Buy & Hold'] = {
                    'full': bh_results,
                    'full_data': data  # Сохраняем данные для расчета количества дней
                }
        except Exception as e:
            print(f"Ошибка при расчете Buy & Hold для {symbol}: {str(e)}")

        for strategy_name, strategy_params in [
            ('Strategy1', Strategy1()),
            ('Strategy2', Strategy2()),
            ('Strategy3', Strategy3()),
            ('Strategy4', Strategy4())
        ]:
            try:
                strategy_results = {}

                # Бэктестирование на Train с гиперпараметрами по умолчанию
                print(f"\n1. Набор данных Train. Начало бэктестирования {strategy_name} с гиперпараметрами по умолчанию: {strategy_params.get_params()}")
                #print(f"Гиперпараметры по умолчанию: {strategy_params.get_params()}")
                train_metrics = backtest_strategy(strategy_params, train_data)
                if train_metrics:
                    strategy_results['train_default'] = train_metrics
                    strategy_results['train_default_params'] = strategy_params.get_params()  # Гиперпараметры
                    strategy_results['train_default_data'] = train_data  # Данные для расчета количества дней

                # Оптимизация гиперпараметров на Train
                print(f"\nНачало оптимизации гиперпараметров для {strategy_name} на Train...")
                param_space = param_spaces[strategy_name]
                best_params = optimize_strategy(globals()[strategy_name], train_data, param_space)
                if best_params:
                    print(f"Оптимальные гиперпараметры для {strategy_name} на Train: {best_params}")
                    optimized_strategy = globals()[strategy_name](**best_params)
                    strategy_results['optimized_params'] = best_params  # Оптимизированные гиперпараметры

                    # Бэктестирование на Test с оптимизированными гиперпараметрами
                    print(f"\n2. Набор данных Test. Начало бэктестирования {strategy_name} с оптимизированными гиперпараметрами на наборе Train: {best_params}")
                    #print(f"Оптимизированные гиперпараметры: {best_params}")
                    test_metrics = backtest_strategy(optimized_strategy, test_data)
                    if test_metrics:
                        strategy_results['test'] = test_metrics
                        strategy_results['test_data'] = test_data  # Данные для расчета количества дней

                    # Объединение Train и Test для оптимизации
                    train_test_data = pd.concat([train_data, test_data])
                    print(f"\nОбъединенный набор данных Train и Test: {train_test_data.index[0].date()} - {train_test_data.index[-1].date()}")

                    # Оптимизация гиперпараметров на объединенном наборе
                    print(f"\nНачало оптимизации гиперпараметров для {strategy_name} на объединенном наборе Train и Test...")
                    best_params_combined = optimize_strategy(globals()[strategy_name], train_test_data, param_space)
                    print(f"Оптимальные гиперпараметры для {strategy_name} на объединенном наборе Train и Test: {best_params_combined}")
                    optimized_strategy_combined = globals()[strategy_name](**best_params_combined)
                    strategy_results['optimized_params_combined'] = best_params_combined  # Оптимизированные гиперпараметры

                    # Бэктестирование на Valid с оптимизированными гиперпараметрами
                    print(f"\n3. Набор данных Valid.  Начало бэктестирования {strategy_name} с оптимизированными гиперпараметрами на объединенном наборе Train и Test: {best_params_combined}")
                    #print(f"Оптимизированные гиперпараметры: {best_params_combined}")
                    valid_metrics = backtest_strategy(optimized_strategy_combined, valid_data)
                    if valid_metrics:
                        strategy_results['valid'] = valid_metrics
                        strategy_results['valid_data'] = valid_data  # Данные для расчета количества дней
                    print("=" * 100)  # Сплошная горизонтальная линия
                if strategy_results:
                    results[strategy_name] = strategy_results

            except Exception as e:
                print(f"Ошибка при анализе стратегии {strategy_name}: {str(e)}")
                continue

        if not results:
            print(f"Не удалось получить результаты для {symbol}")
            return None, None

        return results, data

    except Exception as e:
        print(f"Ошибка при анализе инструмента {symbol}: {str(e)}")
        return None, None


# Функция отображения графика датафрейма
def display_graf(df):
  sns.set(style="darkgrid")
  plt.figure(figsize=(15, 6))
  #display(df)
  dfgr = df
  #dfgr['short_mavg'] = dfgr['Close'].rolling(50).mean()
  #dfgr['long_mavg'] = dfgr['Close'].rolling(200).mean()
  subset = dfgr[dfgr['Ticker'] == 'GOOG']
  plot_data = subset[-500:]
  plt.plot(plot_data.index, plot_data['Close'], label='GOOG')
  #plt.plot(plot_data['short_mavg'], label='50-дневная скользящая средняя')
  #plt.plot(plot_data['long_mavg'], label='200-дневная скользящая средняя')
  plt.title(' ')
  plt.xlabel('Date')
  plt.ylabel('Close Price')
  plt.legend()
  plt.show()


# Функция загрузки данных
def load_data(symbol, start_date='2022-01-01', end_date='2025-10-01'):
    """Загрузка данных с Yahoo Finance"""
    display(f"\nЗагрузка данных с Yahoo Finance {symbol}")
    df = yf.download(symbol, start=start_date, end=end_date)#, interval='1h') progress=False, multi_level_index=False)
    df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
    #df['Ticker'] = symbol

    df = df.dropna()  # Удаление строк с пропущенными значениями
    df = df[~df.index.duplicated()]  # Удаление дубликатов
    df = df.interpolate(method='linear', axis=0).copy()
    #df = df.interpolate(method='time').copy()
    # Сбрасываем мультииндекс, если он есть
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)

    return df


# Функция разделения данных на наборы
def split_data(df):
    """Разделение данных на наборы данных: Train, Test и Valid"""
    train_size = 0.6
    test_size = 0.2
    # Первое разделение на train и остаток
    train_data, temp_data = train_test_split(df, train_size=train_size, shuffle=False)
    # Разделение остатка на test и valid
    test_size_adjusted = test_size / (1 - train_size)
    test_data, valid_data = train_test_split(temp_data, train_size=test_size_adjusted, shuffle=False)
    return train_data, test_data, valid_data



def main():
    """Основная функция с выбором стратегии и визуализацией"""
    all_results = {}
    # Определение списка инструментов
    #symbol = ['GOOG']

    #data = load_data(symbol)

    #if data is None or len(data) < 100:
     #       print(f"Недостаточно данных для {symbol}")
     #       return None, None

    #stock_data = data.copy()
    #stock_data = stock_data.ffill().bfill()

    #if stock_data.isnull().any().any():
    #        print(f"Обнаружены пропущенные значения после обработки в {symbol}")
    #        return None, None

    # Разделение данных на наборы
    #train_data, test_data, valid_data = split_data(stock_data)

    #display(stock_data.head(3))

    #print("\nПериоды наборов данных:")
    #print(f"Train: {train_data.index[0].date()} - {train_data.index[-1].date()}")
    #print(f"Test: {test_data.index[0].date()} - {test_data.index[-1].date()}")
    #print(f"Valid: {valid_data.index[0].date()} - {valid_data.index[-1].date()}")

    #display_graf(test_data)

    #stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'BRK-B', 'NVDA', 'META', 'JPM', 'V', '^GSPC']
    #cryptos = ['SOL-USD', 'ETH-USD', 'BTC-USD', 'XRP-USD', 'USDT-USD']
    stocks = ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'BTC-USD']
    #cryptos = ['SOL-USD', 'BTC-USD']

    instruments = stocks
    for instrument in instruments:

        print(f"\n\033[1;34mАнализ инструмента: {instrument}\033[0m")
        print("=" * 100)  # Сплошная горизонтальная линия
        try:
            results, data = analyze_instrument(instrument)
            if results is not None and data is not None:
                all_results[instrument] = results
                summary_table = create_summary_table(results, instrument)
                display(summary_table)

        except Exception as e:
            print(f"Ошибка в анализе инструмента {instrument}: {str(e)}")
            continue

    # Поиск лучшей стратегии, используя метод отбора
    best_strategy = find_best_strategy(all_results)

    if best_strategy['strategy_name']:
        print(f"\n\033[1;34mОпределение стратегии с лучшими показателями инструменту и по набору данных\033[0m")
        print(f"Лучшая стратегия: {best_strategy['strategy_name']}")
        print(f"Лучший инструмент: {best_strategy['instrument']}")
        print(f"Лучший набор данных: {best_strategy['dataset']}")
        print(f"Доходность с корректировкой риска (Risk-Adjusted Returns): {best_strategy['risk_adjusted_returns']:.2f}")
        print("\nОптимальные гиперпараметры:")
        for param, value in best_strategy['params'].items():
            print(f"{param}: {value}")

        # Вывод метрик производительности
        metrics = best_strategy['metrics']
        print("\nМетрики производительности для лучшей стратегии:")
        print(f"Конечный капитал (final_equity) : {metrics['final_equity']:.2f}")
        print(f"Количество сделок (n_trades): {metrics['n_trades']}")
        print(f"Доходность (returns): {metrics['returns']:.2f}%")
        print(f"Максимальная просадка (max_drawdown): {metrics['max_drawdown']:.2f}%")
        print(f"Процент прибыльных сделок (win_rate): {metrics['win_rate']:.2f}%")
        print(f"Коэффициент Шарпа (sharpe_ratio): {metrics['sharpe_ratio']:.2f}")
        print(f"Коэффициент Сортино (sortino_ratio): {metrics['sortino_ratio']:.2f}")
        print(f"Фактор прибыли (profit_factor): {metrics['profit_factor']:.2f}")

        # Создание объекта стратегии с оптимальными параметрами
        strategy_class = globals()[best_strategy['strategy_name']]
        best_strategy_obj = strategy_class(**best_strategy['params'])

        # Получение набора данных
        data = load_data(best_strategy['instrument'])
        train_data, test_data, valid_data = split_data(data)
        if best_strategy['dataset'] == 'train_default':
            best_data = train_data
        elif best_strategy['dataset'] == 'test':
            best_data = test_data
        else:
            best_data = valid_data

        # Генерация сигналов и проведение бэктестинга
        backtest_strategy(best_strategy_obj, best_data)

        # Анализ сделок
        trades = analyze_trades(best_strategy_obj, best_data, best_strategy_obj.signals)

        # Отображение графика cо сделками
        title = (
            f"Лучшая стратегия со сделками: {best_strategy['strategy_name']} на {best_strategy['instrument']} ({best_strategy['dataset']})"
        )
        fig = plot_best_strategy(
            best_data,
            best_strategy_obj,
            best_strategy_obj.signals,
            best_strategy_obj.equity,
            metrics,
            trades,
            title
        )
        fig.show()

        # Отображение статистики сделок
        print("\nСтатистика сделок для лучшей стратегии:")
        print(f"Всего сделок: {len(trades)}")
        profitable_trades = sum(1 for trade in trades if trade['profit'] > 0)
        print(f"Прибыльных сделок: {profitable_trades} ({profitable_trades/len(trades)*100:.2f}%)")
        print(f"Средняя прибыль на сделку: {sum(trade['profit'] for trade in trades)/len(trades):.2f}%")

        # Отображение графика с сигналами
        title1 = f"Сигналы лучшей стратегии: {best_strategy['strategy_name']} на {best_strategy['instrument']} ({best_strategy['dataset']})"
        fig = plot_best_strategy_signals(
            best_data,
            best_strategy_obj,
            best_strategy_obj.signals,
            metrics,
            title1
        )
        fig.show()

    else:
        print("\nНе найдена лучшая стратегия")

if __name__ == "__main__":
  #https://sky.pro/wiki/profession/indikatory-i-ih-ispolzovanie-v-torgovle-fyuchersami/
  #https://python.ivan-shamaev.ru/example-script-calculate-ta-lib-indicators/
  #https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html
  #https://alchemymarkets.com/education/indicators/on-balance-volume-obv/
  #https://stackoverflow.com/questions/52671594/calculating-stockss-on-balance-volume-obv-in-python#60093990
  #https://trading-strategies.academy/archives/1977
  #https://github.com/TA-Lib/ta-lib-python
  #https://www.tbank.ru/invest/help/educate/trading/about/stochastic/

  main()


Анализ инструмента: AAPL


'\nЗагрузка данных с Yahoo Finance AAPL'

/tmp/ipython-input-2419549224.py:135: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


Размер данных: 939 строк

Периоды наборов данных:
Train: 2022-01-03 - 2024-04-01
Test: 2024-04-02 - 2024-12-27
Valid: 2024-12-30 - 2025-09-30

1. Набор данных Train. Начало бэктестирования Strategy1 с гиперпараметрами по умолчанию: {'ema_period': 20, 'rsi_period': 14, 'macd_fast': 12, 'macd_slow': 26, 'macd_signal': 9}

Начало оптимизации гиперпараметров для Strategy1 на Train...
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [00:01<00:00, 38.11trial/s, best loss: -1.858184620088173]
Оптимальные гиперпараметры для Strategy1 на Train: {'ema_period': np.float64(11.0), 'macd_fast': np.float64(10.0), 'macd_signal': np.float64(5.0), 'macd_slow': np.float64(39.0), 'rsi_period': np.float64(13.0)}

2. Набор данных Test. Начало бэктестирования Strategy1 с оптимизированными гиперпараметрами на наборе Train: {'ema_period': np.float64(11.0), 'macd_fast': np.float64(10.0), 'macd_signal': np.float64(5.0), 'macd_slow': np.float64(39.0), 'rsi_period': np.float64(13.0)}

Объединенный набор данных Train и Test: 2022-01-03 - 2024-12-27

Начало оптимизации гиперпараметров для Strategy1 на объединенном наборе Train и Test...
100%|██████████| 50/50 [00:01<00:00, 27.28trial/s, best loss: -1.9461004005707163]
Оптимальные гиперпараметры для Strategy1 на объединенном наборе Train и Test: {'ema_period': np.float64(18.0), 'macd_fast': np.float64(19.0), 'macd_signal': np.float64(14.0), 'macd_slow': np.float64(24.

...

/usr/local/lib/python3.12/dist-packages/pandas/io/formats/style.py:3811: RuntimeWarning:

invalid value encountered in scalar multiply




Определение стратегии с лучшими показателями инструменту и по набору данных
Лучшая стратегия: Strategy4
Лучший инструмент: GOOGL
Лучший набор данных: train_default
Доходность с корректировкой риска (Risk-Adjusted Returns): 0.00

Оптимальные гиперпараметры:
bb_period: 20
bb_std: 2.0
rsi_period: 14
sma: 10
ema_period: 4
ma_period: 10
macd_fast: 12
macd_slow: 26
macd_signal: 9

Метрики производительности для лучшей стратегии:
Конечный капитал (final_equity) : 1.07
Количество сделок (n_trades): 21
Доходность (returns): 6.54%
Максимальная просадка (max_drawdown): 31.40%
Процент прибыльных сделок (win_rate): 61.90%
Коэффициент Шарпа (sharpe_ratio): 0.25
Коэффициент Сортино (sortino_ratio): 0.06
Фактор прибыли (profit_factor): 1.21


'\nЗагрузка данных с Yahoo Finance GOOGL'

/tmp/ipython-input-2419549224.py:135: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed



Статистика сделок для лучшей стратегии:
Всего сделок: 21
Прибыльных сделок: 13 (61.90%)
Средняя прибыль на сделку: 0.52%
